Date: 05/07/2025

Point of Contact: Abigayle Hodson, Abigayle_Hodson@lbl.gov

Organization: Lawrence Berkeley National Laboratory

Purpose: The purpose of this notebook is to use data from a variety of sources, primarily various releases of the Clean Watersheds Needs Survey (CWNS), to create a dataframe of active wastewater treatment plants in the United States as of 2022. In addition to including flow rate, this dataframe also contains one or more treatment train assignments for each wastewater treatment plant based on *A Guide to Net-Zero Energy Solutions for Water Resource Recovery Facilities* (Tarallo et al., 2015). Treatment train assignments are ultimately used to estimate baseline energy consumption/generation and greenhouse gas emissions on a national scale from wastewater treament facilities.

In [ ]:
#mount google drive - establishes a connection between Google Drive and Colab notebook
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

#import necessary libraries
import pandas as pd
import numpy as np

Mounted at /content/gdrive/


# Create main dataframe of wastewater treatment plants

In [ ]:
def create_wwtp_inventory():
  '''
  Function that creates an inventory of active wwtps in the United States for 2022 using CWNS data
    Returns:
      wwtps_all = dataframe containing the active wwtps and relevant characteristics (ie. location, flow rate, and nutrient removal flags) for 2022
  '''
  #upload facility flow rates from 2022 CWNS
  flow = pd.read_csv('/input_data/FLOW_2022.csv', dtype = {'CWNS_ID':str})

  #filter to total flow
  flow = flow.loc[flow['FLOW_TYPE'] == 'Total Flow']

  #filter to facilities that report non-zero, non-nan flow in 2022
  flow = flow.loc[(flow['CURRENT_DESIGN_FLOW'] != 0) & (~pd.isna(flow['CURRENT_DESIGN_FLOW']))]
  flow.reset_index(inplace = True, drop = True)

  #create dataframe of active wwtps based on facilities that report flow in 2022
  wwtps_all = flow[['CWNS_ID', 'CURRENT_DESIGN_FLOW', 'FUTURE_DESIGN_FLOW']].rename(columns = {'CURRENT_DESIGN_FLOW':'FLOW_2022_MGD', 'FUTURE_DESIGN_FLOW':'FLOW_PROJ_MGD'})

  #upload columns indicating nutrient removal in 2012 (note, 2022 CWNS does not include these columns, so we have to rely on outdated information)
  nutr_rem = pd.read_excel('/input_data/2012_SUMMARY_EFFLUENT.xlsx', sheet_name = 'SUMMARY_EFFLUENT', dtype = {'CWNS_NUMBER':str})
  nutr_rem = nutr_rem[['CWNS_NUMBER','PRES_NITROGEN_REMOVAL','PRES_PHOSPHOROUS_REMOVAL','PRES_AMMONIA_REMOVAL']].rename(columns = {'CWNS_NUMBER':'CWNS_ID'})

  #merge nutrient removal information with main dataframe and rename columns
  wwtps_all = wwtps_all.merge(nutr_rem, on = 'CWNS_ID', how = 'left')

  #upload facility locations
  locations = pd.read_csv('/input_data/2022_FACILITIES.csv', dtype = {'CWNS_ID':str})

  #add state column to main dataframe
  wwtps_all = wwtps_all.merge(locations[['CWNS_ID','STATE_CODE']], on = 'CWNS_ID', how = 'left')
  wwtps_all.rename(columns = {'CWNS_ID':'CWNS_NUM','STATE_CODE':'STATE'}, inplace = True)

  #upload facility types
  types = pd.read_csv('/input_data/2022_FACILITY_TYPES.csv', dtype = {'CWNS_ID':str}).rename(columns= {'CWNS_ID':'CWNS_NUM'})

  #filter to just treatment plants and honey bucket lagoons
  types = types.loc[(types['FACILITY_TYPE'] == 'Treatment Plant') | (types['FACILITY_TYPE'] == 'Honey Bucket Lagoon')].drop_duplicates(subset = 'CWNS_NUM')
  types.reset_index(inplace = True, drop = True)

  #merge facility types with main dataframe to screen out non-treatment plants and honey bucket lagoons from inventory
  wwtps_all = wwtps_all.merge(types[['CWNS_NUM','FACILITY_TYPE']], on = 'CWNS_NUM', how = 'inner')

  #categorize average daily flow rate
  wwtps_all.loc[wwtps_all['FLOW_2022_MGD'] < 2, '2022_FLOW_CAT_MGD'] = 'LESS THAN 2'
  wwtps_all.loc[(wwtps_all['FLOW_2022_MGD'] >= 2) & (wwtps_all['FLOW_2022_MGD'] < 4), '2022_FLOW_CAT_MGD'] = '2 TO 4'
  wwtps_all.loc[(wwtps_all['FLOW_2022_MGD'] >= 4) & (wwtps_all['FLOW_2022_MGD'] < 7), '2022_FLOW_CAT_MGD'] = '4 TO 7'
  wwtps_all.loc[(wwtps_all['FLOW_2022_MGD'] >= 7) & (wwtps_all['FLOW_2022_MGD'] < 16), '2022_FLOW_CAT_MGD'] = '7 TO 16'
  wwtps_all.loc[(wwtps_all['FLOW_2022_MGD'] >= 16) & (wwtps_all['FLOW_2022_MGD'] < 46), '2022_FLOW_CAT_MGD'] = '16 TO 46'
  wwtps_all.loc[(wwtps_all['FLOW_2022_MGD'] >= 46) & (wwtps_all['FLOW_2022_MGD'] < 100), '2022_FLOW_CAT_MGD'] = '46 TO 100'
  wwtps_all.loc[(wwtps_all['FLOW_2022_MGD'] >= 100), '2022_FLOW_CAT_MGD'] = '100 AND ABOVE'

  #categorize projected flow rate
  wwtps_all.loc[wwtps_all['FLOW_PROJ_MGD'] < 2, 'PROJ_FLOW_CAT_MGD'] = 'LESS THAN 2'
  wwtps_all.loc[(wwtps_all['FLOW_PROJ_MGD'] >= 2) & (wwtps_all['FLOW_PROJ_MGD'] < 4), 'PROJ_FLOW_CAT_MGD'] = '2 TO 4'
  wwtps_all.loc[(wwtps_all['FLOW_PROJ_MGD'] >= 4) & (wwtps_all['FLOW_PROJ_MGD'] < 7), 'PROJ_FLOW_CAT_MGD'] = '4 TO 7'
  wwtps_all.loc[(wwtps_all['FLOW_PROJ_MGD'] >= 7) & (wwtps_all['FLOW_PROJ_MGD'] < 16), 'PROJ_FLOW_CAT_MGD'] = '7 TO 16'
  wwtps_all.loc[(wwtps_all['FLOW_PROJ_MGD'] >= 16) & (wwtps_all['FLOW_PROJ_MGD'] < 46), 'PROJ_FLOW_CAT_MGD'] = '16 TO 46'
  wwtps_all.loc[(wwtps_all['FLOW_PROJ_MGD'] >= 46) & (wwtps_all['FLOW_PROJ_MGD'] < 100), 'PROJ_FLOW_CAT_MGD'] = '46 TO 100'
  wwtps_all.loc[(wwtps_all['FLOW_PROJ_MGD'] >= 100), 'PROJ_FLOW_CAT_MGD'] = '100 AND ABOVE'

  #upload EPA regions by state
  epa_regions = pd.read_csv('/input_data/state_EPA_regions.csv', dtype = {'STATE':str})

  #add column for EPA region
  wwtps_all = wwtps_all.merge(epa_regions, on = 'STATE', how = 'left')

  #check for facilities with duplicate entries
  assert wwtps_all['CWNS_NUM'].value_counts().max() == 1

  return wwtps_all

In [ ]:
#create inventory of active wwtps in 2022
wwtps = create_wwtp_inventory()

#add leading zero to CWNS ids with less than 11 digits to ensure correct merge with other datasets
wwtps['CWNS_NUM'] = ['0' + str(cwns) if len(str(cwns)) < 11 else str(cwns) for cwns in wwtps['CWNS_NUM']]

#filter to wwtps in the contiguous United States and reset indexing
wwtps = wwtps.loc[(wwtps['STATE'] != 'PR') & (wwtps['STATE'] != 'AK') & (wwtps['STATE'] != 'VI') & (wwtps['STATE'] != 'HI') & (wwtps['STATE'] != 'MP') & (wwtps['STATE'] != 'GU') & (wwtps['STATE'] != 'AS')]
wwtps.reset_index(inplace = True, drop = True)

In [ ]:
#read in data from the Water Environment Federation's (WEF) biogas database (https://app.powerbi.com/view?r=eyJrIjoiMGFjZDFjZmItMjQ5Yi00ZTlhLWJmNTQtODFiNjlkYjFlODJjIiwidCI6ImI3ZTk3ODAyLTJhNjktNDc3ZS1iN2QyLWY0ZDE2MWMyMTBjYiIsImMiOjF9) to identify wwtps that utilize biogas for electricity generation; note, data was pulled ~2018 before website switched to MS BI
wef_biogas = pd.read_csv(/input_data/WERF_BIOGAS.csv', dtype = {'CWNS_NUM' : str}, encoding = 'latin1')

#change formatting of WEF data from string to binary
wef_biogas.loc[wef_biogas['Electricity_from_combustion-engine'] != 'yes', 'Electricity_from_combustion-engine'] = 0
wef_biogas.loc[wef_biogas['Electricity_from_combustion-engine'] == 'yes', 'Electricity_from_combustion-engine'] = 1
wef_biogas.loc[wef_biogas['Electricity_from_microturbine'] != 'yes', 'Electricity_from_microturbine'] = 0
wef_biogas.loc[wef_biogas['Electricity_from_microturbine'] == 'yes', 'Electricity_from_microturbine'] = 1
wef_biogas.loc[wef_biogas['Electricity_from_turbine'] != 'yes', 'Electricity_from_turbine'] = 0
wef_biogas.loc[wef_biogas['Electricity_from_turbine'] == 'yes', 'Electricity_from_turbine'] = 1
wef_biogas.loc[wef_biogas['Electricity_from_fuelcell'] != 'yes', 'Electricity_from_fuelcell'] = 0
wef_biogas.loc[wef_biogas['Electricity_from_fuelcell'] == 'yes', 'Electricity_from_fuelcell'] = 1
wef_biogas.loc[wef_biogas['Electricity_supplied_to_grid'] != 'yes', 'Electricity_supplied_to_grid'] = 0
wef_biogas.loc[wef_biogas['Electricity_supplied_to_grid'] == 'yes', 'Electricity_supplied_to_grid'] = 1
wef_biogas.loc[wef_biogas['AD'] == 'yes', 'AD'] = 1

#create column that indicates if biogas is used to generate electricity
wef_biogas['biogas_werf'] = wef_biogas['Electricity_from_combustion-engine'] + wef_biogas['Electricity_from_microturbine'] + wef_biogas['Electricity_from_turbine'] + wef_biogas['Electricity_from_fuelcell'] + wef_biogas['Electricity_supplied_to_grid']
wef_biogas.loc[wef_biogas['biogas_werf'] > 0, 'BIOGASELEC_WERF'] = 1

#drop nan values from biogas dataframe to avoid creating duplicates in wwtps dataframe post-merge
wef_biogas = wef_biogas.dropna(subset = 'CWNS_NUM')

#add a leading zero to CWNS ids with a length less than 11 to ensure proper merge
wef_biogas['CWNS_NUM'] = ['0' + str(cwns) if len(str(cwns)) < 11 else str(cwns) for cwns in wef_biogas['CWNS_NUM']]

#create new dataframe that just indicates whether biogas is utilized for electricity generation, according to WEF
biogas_wef = wef_biogas[['CWNS_NUM','BIOGASELEC_WERF']].dropna()

#merge biogas info from WEF with faciltiies info from CWNS
wwtps = pd.merge(left = wwtps, right = biogas_wef, how = 'left', on = 'CWNS_NUM')

#replace nan values with zeros
wwtps['BIOGASELEC_WERF'] = wwtps['BIOGASELEC_WERF'].fillna(0)

In [ ]:
#read in data from the Department of Energy's Combined Heat and Power Installation database (https://doe.icfwebservices.com/chp) to identify wwtps that utilize biogas for electricity generation
doe_biogas = pd.read_csv(/input_data/doe_chpdb-WWTP.csv', dtype = {'CWNS_NUM' : str}, encoding = 'latin1')

#drop duplicate and nan values from biogas dataframe to avoid creating duplicates in wwtps dataframe post-merge
doe_biogas.dropna(subset = 'CWNS_NUM', inplace = True)

#add a leading zero to old CWNS ids with a length less than 11 to ensure proper merge
doe_biogas['CWNS_NUM'] = ['0' + str(cwns) if len(str(cwns)) < 11 else str(cwns) for cwns in doe_biogas['CWNS_NUM']]
doe_biogas.drop_duplicates(subset = 'CWNS_NUM', inplace = True)

#create a dataframe that just indicates whether biogas is used for electricity generation, according to DOE
biogas_doe = doe_biogas[['CWNS_NUM','BIOGAS_DOE_2022']]

#merge biogas info from DOE with faciltiies info from CWNS
wwtps = pd.merge(left = wwtps, right = biogas_doe, how = 'left', on = 'CWNS_NUM')

#replace nan values with zeros
wwtps['BIOGAS_DOE_2022'] = wwtps['BIOGAS_DOE_2022'].fillna(0)

#create column that indicate whether biogas was used for electricity generation based on the results of both DOE and WEF
wwtps['BIOGAS_EL_2022'] = 0
wwtps.loc[((wwtps['BIOGASELEC_WERF'] + wwtps['BIOGAS_DOE_2022']) > 0), 'BIOGAS_EL_2022'] = 1

# Create cumulative unit process list for wastewater treatment plants

In [ ]:
#read in unit processes from the 2022 CWNS
up2022 = pd.read_csv(/input_data/2022_SUMMARY_UNIT_PROCESSES.csv', dtype = {"CWNS_ID" : str})
up2022.rename(columns = {'CWNS_ID':'CWNS_NUM'}, inplace = True)

#add a leading zero to CWNS ids with a length less than 11 to ensure proper merge
up2022['CWNS_NUM'] = ['0' + str(cwns) if len(str(cwns)) < 11 else str(cwns) for cwns in up2022['CWNS_NUM']]

#change formatting of 2022 unit process names to match that of prior years
#note: 'Biological Treatment, Other' was manually corrected to be more specific. 'Chemical N Removal' was assumed to be roughly the same energy intensity as 'Chemical P removal'
upnames_2022 = pd.read_csv(/input_data/UNIT_PROCESS_NAMES_2022.csv')
up2022 = pd.merge(left = up2022, right = upnames_2022, how = 'left', left_on = 'UNIT_PROCESS', right_on = '2022_UNIT_PROCESS_NAME')

#filter to relevant columns and rename to match the formatting of old unit process dataframes
up2022 = up2022[['CWNS_NUM','FINAL_UNIT_PROCESS_NAME','EXISTING_FLAG','PLANNED_FLAG']]
up2022.rename(columns = {'EXISTING_FLAG':'PRES_IND','PLANNED_FLAG':'PROJ_IND'}, inplace = True)
up2022.loc[up2022['PRES_IND'] == 'Y', 'PRES_IND'] = 1
up2022.loc[up2022['PRES_IND'] == 'N', 'PRES_IND'] = 0
up2022.loc[pd.isna(up2022['PRES_IND']), 'PRES_IND'] = 0
up2022.loc[up2022['PROJ_IND'] == 'Y', 'PROJ_IND'] = 1
up2022.loc[up2022['PROJ_IND'] == 'N', 'PROJ_IND'] = 0
up2022.loc[pd.isna(up2022['PROJ_IND']), 'PROJ_IND'] = 0
up2022['REPORT_YEAR'] = 2022

#read in unit processs reported in the 2004, 2008, and 2012 releases of CWNS
up2012 = pd.read_csv(/input_data/2012_SUMMARY_UNIT_PROCESS.csv', dtype = {'REPORT_YEAR':int, "CWNS_NUMBER":str, "TREATMENT_TYPE":str,"UNIT_PROCESS":str}, encoding = 'latin1')
up2008 = pd.read_csv(/input_data/2008_SUMMARY_UNIT_PROCESS.csv',dtype = {'REPORT_YEAR':int, "CWNS_NUMBER":str, "TREATMENT_TYPE":str,"UNIT_PROCESS":str}, encoding = 'latin1')
up2004 = pd.read_csv(/input_data/2004_Unit_Processes.csv', dtype = {'REPORT_YEAR':int, "CWNS_NUMBER":str, "TREATMENT_TYPE":str,"UNIT_PROCESS":str}, encoding = 'latin1')

#aggregate 2004, 2008, and 2012 unit process lists
up_old = pd.concat([up2012, up2008,up2004], axis = 0)
up_old.drop(['BACKUP_IND','PLANNED_YEAR','ADDITIONAL_NOTES','LAST_UPDATED_TS','BLANK','CHANGE_TYPE_CAT','SORT_SEQUENCE','KEEP_UP_CODE', 'CHGTP_NAME_CAT','TREATMENT_TYPE','Notes'], inplace = True, axis = 1)
up_old.rename(columns = {'CWNS_NUMBER':'CWNS_NUM'}, inplace = True)

#add a leading zero to CWNS ids with a length less than 11 to ensure proper merge
up_old['CWNS_NUM'] = ['0' + str(cwns) if len(str(cwns)) < 11 else str(cwns) for cwns in up_old['CWNS_NUM']]

#reconcile unit process naming conventions between report years
upnames = pd.read_csv(/input_data/UNIT_PROCESS_NAMES.csv')
up_old = pd.merge(left = up_old, right = upnames, how = 'left', left_on = 'UNIT_PROCESS', right_on = 'ORIGINAL_UP_NAME')
up_old.drop(['ORIGINAL_UP_NAME'], inplace = True, axis = 1)

#remove processes listed for abandoment in 2004, 2008, or 2012 and processes listed as both PRES_IND = N and PROJ_IND = N
up_old = up_old.loc[up_old['CHANGE_TYPE'] != 'Abandonment']
up_old = up_old.loc[~((up_old['PRES_IND'] == 'N') & (up_old['PRES_IND'] == 'N'))]
up_old = up_old[['CWNS_NUM','REPORT_YEAR','PRES_IND','PROJ_IND','FINAL_UNIT_PROCESS_NAME']]

#change formatting of present and projected indices to binary
up_old.loc[up_old['PRES_IND'] == 'Y', 'PRES_IND'] = 1
up_old.loc[up_old['PRES_IND'] == 'N', 'PRES_IND'] = 0
up_old.loc[up_old['PROJ_IND'] == 'Y', 'PROJ_IND'] = 1
up_old.loc[up_old['PROJ_IND'] == 'N', 'PROJ_IND'] = 0

#join 2022 unit process list and old unit process list
uplist_all = pd.concat([up2022, up_old], axis = 0)

#sort by CWNS ID and reporting year
uplist_all.sort_values(by = ['CWNS_NUM','REPORT_YEAR'], ascending = True, inplace = True)

#drop duplicate unit processes and keep most recent entry
uplist_all.drop_duplicates(subset = ['CWNS_NUM', 'FINAL_UNIT_PROCESS_NAME','PRES_IND','PROJ_IND'], inplace = True, keep = 'last')
uplist_recent = uplist_all.reset_index(drop = True)

<ipython-input-101-0bf849682014>:27: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  up2004 = pd.read_csv('/content/gdrive/MyDrive/AMO/baselining_paper/treatment_train_assignment/input_data/2004_Unit_Processes.csv', dtype = {'REPORT_YEAR':int, "CWNS_NUMBER":str, "TREATMENT_TYPE":str,"UNIT_PROCESS":str}, encoding = 'latin1')


In [ ]:
#use supplementary biogas databases to add anaerobic digestion to unit process list for facilities flagged as using biogas for electricity production
wef_biogas_ad = wef_biogas[['CWNS_NUM', 'AD']].drop_duplicates(subset = 'CWNS_NUM')
wef_biogas_ad.loc[wef_biogas_ad['AD'] == 1, 'FINAL_UNIT_PROCESS_NAME'] = 'Biosolids Anaerobic Digestion, Other'
wef_biogas_ad['REPORT_YEAR'] = 2013
wef_biogas_ad['PRES_IND'] = 1
wef_biogas_ad['PROJ_IND'] = 1
wef_biogas_ad = wef_biogas_ad[['CWNS_NUM','FINAL_UNIT_PROCESS_NAME','REPORT_YEAR','PRES_IND','PROJ_IND']]

doe_biogas_ad = doe_biogas.loc[doe_biogas['BIOGAS_DOE_2022'] == 1][['CWNS_NUM','Last Verified']]
doe_biogas_ad.rename(columns = {'Last Verified':'REPORT_YEAR'}, inplace = True)
doe_biogas_ad['FINAL_UNIT_PROCESS_NAME'] = 'Biosolids Anaerobic Digestion, Other'
doe_biogas_ad['PRES_IND'] = 1
doe_biogas_ad['PROJ_IND'] = 1

#merge additional anaerobic digestion processes with cumulative unit process list
uplist_recent = pd.concat([uplist_recent, wef_biogas_ad, doe_biogas_ad], axis = 0, ignore_index = False)

In [ ]:
#assign key unit processes a code (ie. 'Activated Sludge' is assigned the code 'AS'); note, not all unit processes receive a code
up_eicodes = pd.read_csv(/input_data/UNIT_PROCESS_EI_CODES_WERF.csv')
uplist_eicodes = uplist_recent.merge(up_eicodes[['FINAL_UNIT_PROCESS_NAME','WERF_CODE','DISPOSAL_CODE']].drop_duplicates(subset = ['FINAL_UNIT_PROCESS_NAME']), how = 'left', on = 'FINAL_UNIT_PROCESS_NAME')

#create column to indicate if a unit process was present in 2022
uplist_eicodes['2022_MIN_IND'] = uplist_eicodes['PRES_IND']

In [ ]:
#manual corrections to cumulative unit process list (Christina Polcuch, 2023) for large facilities that were initially assigned multiple treatment trains
#here, we conducted manual verification of unit processes for selected facilities using publicly available information. Relevant unit processes were updated accordingly in the unit process list, and the note "Corrected based on manual check of large facilities with multiple treatment train assignments (2023)" was added in the UP_ID_NOTE column.

#fix Lewiston, ME; no nutrient removal
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '23000011001') & ((uplist_eicodes['WERF_CODE'] == 'AS-A2O')), '2022_MIN_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '23000011001') & ((uplist_eicodes['WERF_CODE'] == 'AS-A2O')), 'PRES_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '23000011001') & ((uplist_eicodes['WERF_CODE'] == 'AS-A2O')), 'UP_ID_NOTE'] = 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'

#fix Brockton, MA; no TF, no incineration
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '25000024001') & ((uplist_eicodes['WERF_CODE'] == 'TF')), '2022_MIN_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '25000024001') & ((uplist_eicodes['WERF_CODE'] == 'TF')), 'PROJ_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '25000024001') & ((uplist_eicodes['WERF_CODE'] == 'TF')), 'UP_ID_NOTE'] = 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '25000024001') & ((uplist_eicodes['WERF_CODE'] == 'MHI')), '2022_MIN_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '25000024001') & ((uplist_eicodes['WERF_CODE'] == 'MHI')), 'PROJ_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '25000024001') & ((uplist_eicodes['WERF_CODE'] == 'MHI')), 'UP_ID_NOTE'] = 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'

#fix GLWA plants; no phosphorus or nutrient removal
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '26000569001') & ((uplist_eicodes['WERF_CODE'] == 'AS-A2O')), '2022_MIN_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '26000569001') & ((uplist_eicodes['WERF_CODE'] == 'AS-A2O')), 'PRES_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '26000569001') & ((uplist_eicodes['WERF_CODE'] == 'AS-A2O')), 'UP_ID_NOTE'] = 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'

#fix WY WWTP; no trickling filter
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '26000334001') & ((uplist_eicodes['WERF_CODE'] == 'TF')), '2022_MIN_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '26000334001') & ((uplist_eicodes['WERF_CODE'] == 'TF')), 'PRES_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '26000334001') & ((uplist_eicodes['WERF_CODE'] == 'TF')), 'UP_ID_NOTE'] = 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'

#fix Mcalpine Creek WWTP; no BNIT, LAGOON_AER, NIT, or TF; add biogas utilization
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '37006001002') & ((uplist_eicodes['WERF_CODE'] == 'BNIT')), '2022_MIN_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '37006001002') & ((uplist_eicodes['WERF_CODE'] == 'BNIT')), 'PRES_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '37006001002') & ((uplist_eicodes['WERF_CODE'] == 'BNIT')), 'UP_ID_NOTE'] = 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '37006001002') & ((uplist_eicodes['WERF_CODE'] == 'LAGOON_AER')), '2022_MIN_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '37006001002') & ((uplist_eicodes['WERF_CODE'] == 'LAGOON_AER')), 'PRES_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '37006001002') & ((uplist_eicodes['WERF_CODE'] == 'LAGOON_AER')), 'UP_ID_NOTE'] = 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '37006001002') & ((uplist_eicodes['WERF_CODE'] == 'NIT')), '2022_MIN_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '37006001002') & ((uplist_eicodes['WERF_CODE'] == 'NIT')), 'PRES_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '37006001002') & ((uplist_eicodes['WERF_CODE'] == 'NIT')), 'UP_ID_NOTE'] = 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '37006001002') & ((uplist_eicodes['WERF_CODE'] == 'TF')), '2022_MIN_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '37006001002') & ((uplist_eicodes['WERF_CODE'] == 'TF')), 'PRES_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '37006001002') & ((uplist_eicodes['WERF_CODE'] == 'TF')), 'UP_ID_NOTE'] = 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'
wwtps.loc[wwtps['CWNS_NUM'] == '37006001002', 'BIOGAS_EL_2022'] = 1

#fix NEORSD Westerly WWTP; no AND, AS, or CHEM-P
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '39001666003') & ((uplist_eicodes['WERF_CODE'] == 'AND')), '2022_MIN_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '39001666003') & ((uplist_eicodes['WERF_CODE'] == 'AND')), 'PRES_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '39001666003') & ((uplist_eicodes['WERF_CODE'] == 'AND')), 'UP_ID_NOTE'] = 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '39001666003') & ((uplist_eicodes['WERF_CODE'] == 'AS')), '2022_MIN_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '39001666003') & ((uplist_eicodes['WERF_CODE'] == 'AS')), 'PRES_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '39001666003') & ((uplist_eicodes['WERF_CODE'] == 'AS')), 'UP_ID_NOTE'] = 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '39001666003') & ((uplist_eicodes['WERF_CODE'] == 'CHEM-P')), '2022_MIN_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '39001666003') & ((uplist_eicodes['WERF_CODE'] == 'CHEM-P')), 'PRES_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '39001666003') & ((uplist_eicodes['WERF_CODE'] == 'CHEM-P')), 'UP_ID_NOTE'] = 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'

#fix Hopewell Regional WWTP; no FBI, add MHI
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '51000238001') & ((uplist_eicodes['WERF_CODE'] == 'FBI')), '2022_MIN_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '51000238001') & ((uplist_eicodes['WERF_CODE'] == 'FBI')), 'PRES_IND'] = 0
uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == '51000238001') & ((uplist_eicodes['WERF_CODE'] == 'FBI')), 'UP_ID_NOTE'] = 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'
hopewell_idx = uplist_eicodes[uplist_eicodes['CWNS_NUM'] == '51000238001'].index.max()
hopewell_add = pd.Series({'CWNS_NUM': '51000238001', 'WERF_CODE': 'MHI', '2022_MIN_IND': 1, 'PRES_IND': 1, 'FINAL_UNIT_PROCESS_NAME': 'Biosolids Incineration, Multiple Hearth', 'UP_ID_NOTE': 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'}).to_frame().T
uplist_eicodes = pd.concat([uplist_eicodes.iloc[:hopewell_idx], hopewell_add, uplist_eicodes.iloc[hopewell_idx:]], ignore_index=True)

#fix Arlington, CO WPCP; add LIME
arlington_idx = uplist_eicodes[uplist_eicodes['CWNS_NUM'] == '51000319001'].index.max()
arlington_add = pd.Series({'CWNS_NUM': '51000319001', 'WERF_CODE': 'LIME', '2022_MIN_IND': 1, 'PRES_IND': 1, 'FINAL_UNIT_PROCESS_NAME': 'Biosolids Lime Stabilization', 'UP_ID_NOTE': 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'}).to_frame().T
uplist_eicodes = pd.concat([uplist_eicodes.iloc[:arlington_idx], arlington_add, uplist_eicodes.iloc[arlington_idx:]], ignore_index=True)

In [ ]:
#manual updates to add and remove lagoons (Christina Polcuch, 2023)
#here, we confirmed the presence of lagoons at facilities larger than 10 MGD using publicly available information

#import list of lagoons to add from EPA lagoon inventory and manual checks
lagoon_add = pd.read_csv(/input_data/cwns_lagoon_add_ttrains_info.csv', dtype = {'CWNS_NUM':str})

#add leading zeros to CWNS ids to ensure proper merge with other datasets
lagoon_add['CWNS_NUM'] = ['0' + str(cwns) if len(str(cwns)) < 11 else str(cwns) for cwns in lagoon_add['CWNS_NUM']]

#add in columns for concatenation with main uplist_eicodes dataframe
lagoon_add['REPORT_YEAR'] = 2022
lagoon_add['UP_ID_NOTE'] = 'Assigned using updated data on the presence of lagoons from EPA (2022)'
lagoon_add['2022_MIN_IND'] = 1
lagoon_add['PRES_IND'] = 1
lagoon_add.rename(columns = {'LAGOON_CODE':'WERF_CODE','LAGOON_NAME':'FINAL_UNIT_PROCESS_NAME'}, inplace = True)

#concatenate dataframe which contains additional lagoons found in EPA survey/manual checks and main unit process list dataframe
uplist_eicodes = pd.concat([uplist_eicodes, lagoon_add], axis = 0)

#assume lagoons that were added are projected to remain
uplist_eicodes['PROJ_IND'] = uplist_eicodes['PROJ_IND'].fillna(1)

#drop duplicates unit processes to ensure most recent lagoons are kept in final unit process list
uplist_eicodes.sort_values(by = ['CWNS_NUM','REPORT_YEAR'], ascending = True, inplace = True)
uplist_eicodes.drop_duplicates(subset = ['CWNS_NUM','WERF_CODE','DISPOSAL_CODE','PRES_IND','PROJ_IND'], inplace = True, keep = 'last')
uplist_eicodes.reset_index(inplace = True, drop = True)

#import list of lagoons to remove based on manual checks
lagoon_removed = pd.read_csv(/input_data/cwns_lagoon_remove.csv', dtype = {'CWNS_NUM': str})

#add leading zeros to CWNS ids to ensure proper match with uplist_eicodes
lagoon_removed['CWNS_NUM'] = ['0' + str(cwns) if len(str(cwns)) < 11 else str(cwns) for cwns in lagoon_removed['CWNS_NUM']]

#loop through lagoons that need to be removed
for index, row in lagoon_removed.iterrows():
    #get the CWNS number and lagoon code to remove
    lagoon_removed_CWNS = row['CWNS_NUM']
    lagoon_removed_code = row['REMOVE']

    #search for the row in uplist_eicodes where the CWNS number matches the CWNS number in lagoon_removed
    CWNS_match_row = uplist_eicodes.loc[(uplist_eicodes['CWNS_NUM'] == lagoon_removed_CWNS) & (uplist_eicodes['WERF_CODE'].str.contains(lagoon_removed_code))]

    #if a matching row is found, set '2022_MIN_IND', '2022_MAX_IND', and 'PROJ_IND' equal to 0 and update 'UP_ID_NOTE'
    if not CWNS_match_row.empty:
        uplist_eicodes.loc[CWNS_match_row.index, ['2022_MIN_IND','PROJ_IND']] = 0
        uplist_eicodes.loc[CWNS_match_row.index, 'UP_ID_NOTE'] = 'Corrected based on manual check of large lagoons (2023/2024)'

<ipython-input-105-5e8c8eda376f>:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  uplist_eicodes['PROJ_IND'] = uplist_eicodes['PROJ_IND'].fillna(1)


In [ ]:
#identify plants that have FBI, MHI, LAND_APP, or LANDFILL as a biosolids disposal method in 2022 and export for use in biosolids emissions calculations
disposal_2022 = uplist_eicodes[['CWNS_NUM','REPORT_YEAR','PRES_IND','DISPOSAL_CODE']]
disposal_2022 = disposal_2022.loc[(disposal_2022['PRES_IND'] == 1)]
disposal_2022 = disposal_2022.dropna(subset = ['DISPOSAL_CODE'])
disposal_2022.sort_values(by = ['CWNS_NUM','REPORT_YEAR'], ascending = False, inplace = True, ignore_index = True)
disposal_2022.drop_duplicates(subset = 'CWNS_NUM', inplace = True, ignore_index = False, keep = "first")
disposal_2022_pvt = pd.pivot_table(disposal_2022, index = 'CWNS_NUM', values = 'PRES_IND',  columns = 'DISPOSAL_CODE', aggfunc = np.sum, fill_value = 0)
disposal_2022_pvt.to_csv(/output_data/disposal_2022.csv')

<ipython-input-106-6d4f32a4413e>:7: FutureWarning: The provided callable <function sum at 0x7f129510cf40> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  disposal_2022_pvt = pd.pivot_table(disposal_2022, index = 'CWNS_NUM', values = 'PRES_IND',  columns = 'DISPOSAL_CODE', aggfunc = np.sum, fill_value = 0)
<ipython-input-106-6d4f32a4413e>:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  disposal_2022_pvt = pd.pivot_table(disposal_2022, index = 'CWNS_NUM', values = 'PRES_IND',  columns = 'DISPOSAL_CODE', aggfunc = np.sum, fill_value = 0)


In [ ]:
#drop unit processes that do not have an associated WERF code, as these are not necessary to form treatment train assignments
uplist_eicodes.dropna(subset = 'WERF_CODE', inplace = True)

In [ ]:
#manual check for nutrient removal (Christina Polcuch, 2023)
#for facilities where the cumulative unit process list was revised based on publicly available information, correct the nutrient removal flags in the main wwtps dataframe to remain consistent with uplist_eicodes

#fix Lewiston, ME; remove nutrient removal
wwtps.loc[wwtps['CWNS_NUM'] == '23000011001', 'PRES_NITROGEN_REMOVAL'] = 0
wwtps.loc[wwtps['CWNS_NUM'] == '23000011001', 'PRES_PHOSPHOROUS_REMOVAL'] = 0
wwtps.loc[wwtps['CWNS_NUM'] == '23000011001', 'PRES_AMMONIA_REMOVAL'] = 0
uplist_eicodes.loc[uplist_eicodes['CWNS_NUM'] == '23000011001', 'UP_ID_NOTE'] = 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'

#fix GLWA plants; no phosphorus or nutrient removal yet
wwtps.loc[wwtps['CWNS_NUM'] == '26000569001', 'PRES_NITROGEN_REMOVAL'] = 0
wwtps.loc[wwtps['CWNS_NUM'] == '26000569001', 'PRES_PHOSPHOROUS_REMOVAL'] = 0
wwtps.loc[wwtps['CWNS_NUM'] == '26000569001', 'PRES_AMMONIA_REMOVAL'] = 0
uplist_eicodes.loc[uplist_eicodes['CWNS_NUM'] == '26000569001', 'UP_ID_NOTE'] = 'Corrected based on manual check of large facilities with multiple treatment train assignments (2023)'

In [ ]:
#create table with manually corrected wwtps to later add UP_ID_NOTE column to treatment train assignment dataframe
manual_check_ups = uplist_eicodes[['CWNS_NUM','UP_ID_NOTE']]
manual_check_ups = manual_check_ups.dropna()
manual_check_ups = manual_check_ups.drop_duplicates(subset = 'CWNS_NUM')

# Define functions for treatment train assignment

In [ ]:
def clear_old_treatment(uplist_yr_table, scenario):
  '''
  Function that removes outdated secondary/solids processes from cumulative unit process list
    Parameters:
      uplist_yr_table = dataframe of all reported unit processes relevant to treatment train assignment
      scenario = scenario for treatment train assignment (e.g. 2022_MIN = treatment trains anticipated to have been active in 2022)
    Returns:
      uplist_werf_yr_final = modified dataframe of reported unit processes relevant to treatment train assignment, excluding old secondary/solids treatment processes
  '''
  #extract all solids treatment processes from cumulative unit process list
  uplist_yr_table_dig = uplist_yr_table.loc[(uplist_yr_table['WERF_CODE'] == 'AED') | (uplist_yr_table['WERF_CODE'] == 'AND') | (uplist_yr_table['WERF_CODE'] == 'LIME') | (uplist_yr_table['WERF_CODE'] == 'FBI') | (uplist_yr_table['WERF_CODE'] == 'MHI') | (uplist_yr_table['WERF_CODE'] == 'BIODRY') | (uplist_yr_table['WERF_CODE'] == 'BS_LAGOON')]

  #identify facilities with more than one reported solids process
  uplist_yr_table_dig['DUP'] = uplist_yr_table_dig.duplicated(subset = 'CWNS_NUM', keep = False)
  uplist_yr_table_dig_dup = uplist_yr_table_dig.loc[(uplist_yr_table_dig['DUP'] == True)]

  #identify most recently reported solids process
  up_werf_dig_dup_maxyr = uplist_yr_table_dig_dup.groupby(['CWNS_NUM'])['REPORT_YEAR'].describe()[['max']]
  uplist_yr_table_dig_dup_keep = pd.merge(left = uplist_yr_table_dig_dup, right = up_werf_dig_dup_maxyr, how = 'left', on = 'CWNS_NUM')
  uplist_yr_table_dig_dup_keep.loc[(uplist_yr_table_dig_dup_keep['REPORT_YEAR'] == uplist_yr_table_dig_dup_keep['max']), 'KEEP'] = 1
  uplist_yr_table_dig_dup_keep.loc[(uplist_yr_table_dig_dup_keep['REPORT_YEAR'] != uplist_yr_table_dig_dup_keep['max']), 'KEEP'] = 0
  uplist_yr_table_dig_dup_keep = uplist_yr_table_dig_dup_keep.loc[:,['CWNS_NUM','REPORT_YEAR','WERF_CODE','KEEP']]
  uplist_yr_table_dig_dup_keep.sort_values(by = ['CWNS_NUM','REPORT_YEAR'], ascending=False, inplace=True, ignore_index=True)

  #remove less recently reported solids processes from cumulative unit process list
  uplist_werf_yr_cut = pd.merge(left = uplist_yr_table, right = uplist_yr_table_dig_dup_keep, how = 'left', on = ['CWNS_NUM','REPORT_YEAR','WERF_CODE'])
  uplist_werf_yr_cut = uplist_werf_yr_cut.loc[:,['CWNS_NUM','REPORT_YEAR',f"{scenario}_IND",'WERF_CODE','KEEP']]
  uplist_werf_yr = uplist_werf_yr_cut.loc[(uplist_werf_yr_cut['KEEP'] != 0)]
  uplist_werf_yr = uplist_werf_yr.loc[:,['CWNS_NUM','REPORT_YEAR',f"{scenario}_IND",'WERF_CODE']]

  #extract all secondary treatment processes from cumulative unit process list, excluding biogas utilization, biosolids lagoons, and polishing lagoons
  uplist_werf_yr_sec = uplist_werf_yr.loc[(uplist_werf_yr['WERF_CODE'].str.contains('AS')) | (uplist_werf_yr['WERF_CODE'].str.contains('TF')) | (uplist_werf_yr['WERF_CODE'].str.contains('POND')) | (uplist_werf_yr['WERF_CODE'].str.contains('LAGOON'))]
  uplist_werf_yr_sec = uplist_werf_yr_sec.loc[(uplist_werf_yr_sec['WERF_CODE'] != 'BIOGAS_CWNS')]
  uplist_werf_yr_sec = uplist_werf_yr_sec.loc[(uplist_werf_yr_sec['WERF_CODE'] != 'BS_LAGOON')]
  uplist_werf_yr_sec = uplist_werf_yr_sec.loc[(uplist_werf_yr_sec['WERF_CODE'] != 'LAGOON_POL')]

  #identify facilities with more than one reported secondary process
  uplist_werf_yr_sec['DUP'] = uplist_werf_yr_sec.duplicated(subset = 'CWNS_NUM', keep=False)
  uplist_werf_yr_sec_dup = uplist_werf_yr_sec.loc[(uplist_werf_yr_sec['DUP'] == True)]

  #identify most recently reported secondary process
  up_werf_sec_dup_maxyr = uplist_werf_yr_sec_dup.groupby(['CWNS_NUM'])['REPORT_YEAR'].describe()[['max']]
  uplist_werf_yr_sec_dup_keep = pd.merge(left = uplist_werf_yr_sec_dup, right = up_werf_sec_dup_maxyr, how = 'left', on = 'CWNS_NUM')
  uplist_werf_yr_sec_dup_keep.loc[(uplist_werf_yr_sec_dup_keep['REPORT_YEAR'] == uplist_werf_yr_sec_dup_keep['max']), 'KEEP'] = 1
  uplist_werf_yr_sec_dup_keep.loc[(uplist_werf_yr_sec_dup_keep['REPORT_YEAR'] != uplist_werf_yr_sec_dup_keep['max']), 'KEEP'] = 0
  uplist_werf_yr_sec_dup_keep = uplist_werf_yr_sec_dup_keep.loc[:,['CWNS_NUM','REPORT_YEAR','WERF_CODE','KEEP']]
  uplist_werf_yr_sec_dup_keep.sort_values(by = ['CWNS_NUM','REPORT_YEAR'], ascending = False, inplace = True, ignore_index = True)

  #remove less recently reported secondary processes from cumulative unit process list
  uplist_werf_yr_cut2 = pd.merge(left = uplist_werf_yr, right = uplist_werf_yr_sec_dup_keep, how = 'left', left_on = ['CWNS_NUM', 'REPORT_YEAR', 'WERF_CODE'], right_on = ['CWNS_NUM','REPORT_YEAR','WERF_CODE'])
  uplist_werf_yr_cut2 = uplist_werf_yr_cut2.loc[:,['CWNS_NUM','REPORT_YEAR',f"{scenario}_IND",'WERF_CODE','KEEP']]
  uplist_werf_yr_cut2 = uplist_werf_yr_cut2.loc[(uplist_werf_yr_cut2['KEEP'] != 0)]
  uplist_werf_yr_final = uplist_werf_yr_cut2.loc[:,['CWNS_NUM','REPORT_YEAR',f"{scenario}_IND",'WERF_CODE']]
  uplist_werf_yr_final.drop_duplicates(subset = ['CWNS_NUM','WERF_CODE'], inplace=True, ignore_index=True)

  return uplist_werf_yr_final

In [ ]:
def unit_process_pivot(uplist_werf_yr_to_pivot, scenario):
  '''
  Function that groups unit processes treated equivalently in Tarallo et al., 2015, then converts cumulative unit process list into a pivot table
    Parameters:
      uplist_werf_yr_to_pivot = cumulative unit process list
      scenario = scenario for treatment train assignment (e.g. 2022_MIN = treatment trains anticipated to have been active in 2022)
    Returns:
      tt_uppvt_yr = pivot table of active unit processes in 2022 for each wastewater treatment plant, grouped when applicable
  '''

  #create pivot table of unit processes
  tt_uppvt_yr = pd.pivot_table(uplist_werf_yr_to_pivot, index = 'CWNS_NUM', values = "%s_IND" %scenario, columns = 'WERF_CODE', aggfunc = np.sum, fill_value = 0)

  #add empty columns for unit processes that were not reported in selected year
  all_ups = up_eicodes['WERF_CODE'].unique()
  reported_ups = uplist_werf_yr_to_pivot['WERF_CODE'].unique()
  for up in all_ups:
    if up not in reported_ups:
      tt_uppvt_yr[up] = 0

  #group conventional activated sludge without nutrient removal
  tt_uppvt_yr['SUM_AS'] = tt_uppvt_yr['AS'] + tt_uppvt_yr['AS-A2O'] + tt_uppvt_yr['AS-BDENIT'] + tt_uppvt_yr['AS-EA'] + tt_uppvt_yr['AS-P'] + tt_uppvt_yr['AS-PUREO'] + tt_uppvt_yr['AS-SA']
  tt_uppvt_yr['BASIC_AS'] = tt_uppvt_yr['AS'] + tt_uppvt_yr['AS-EA'] + tt_uppvt_yr['AS-SA'] + tt_uppvt_yr['AS-OD'] + tt_uppvt_yr['AS-SBR']
  tt_uppvt_yr.loc[tt_uppvt_yr['BASIC_AS'] > 0, 'BASIC_AS'] = 1

  #group all activated sludge with nitrogen removal
  tt_uppvt_yr['AS_BNR_N'] = tt_uppvt_yr['AS-A2O'] + tt_uppvt_yr['AS-BDENIT']
  tt_uppvt_yr.loc[tt_uppvt_yr['AS_BNR_N'] > 0, 'AS_BNR_N'] = 1
  tt_uppvt_yr.loc[(tt_uppvt_yr['AS'] + tt_uppvt_yr['BNR']) > 1, 'AS_BNR_N'] = 1

  #group all trickling filters
  tt_uppvt_yr['TF_ALL'] = tt_uppvt_yr['TF'] + tt_uppvt_yr['TF-BF'] + tt_uppvt_yr['TF-RBC']
  tt_uppvt_yr.loc[tt_uppvt_yr['TF_ALL'] > 0, 'TF_ALL'] = 1

  #group all biological phosphorous removal
  tt_uppvt_yr.loc[(((tt_uppvt_yr['SUM_AS'] > 0) & (tt_uppvt_yr['BIO-P'] > 0)) | (tt_uppvt_yr['AS-P'] == 1)), 'AS_BNR_P'] = 1

  #override multiple entries to just one
  tt_uppvt_yr.loc[tt_uppvt_yr['PRIMARY'] > 0, 'PRIMARY'] = 1
  tt_uppvt_yr.loc[tt_uppvt_yr['MHI'] > 0, 'MHI'] = 1
  tt_uppvt_yr.loc[tt_uppvt_yr['BDENIT'] > 0, 'BDENIT'] = 1

  #drop unnecessary columns and fill nan values with zero
  tt_uppvt_yr = tt_uppvt_yr.drop(['DEWATER', 'DISINF','DISINF-UV','LAGOON_POL'], axis = 1)
  tt_uppvt_yr = tt_uppvt_yr.fillna(0)

  #add column with the total number of unit processes used for each wwtp
  tt_uppvt_yr['COUNT_UP'] = tt_uppvt_yr['AED'] + tt_uppvt_yr['AND'] + tt_uppvt_yr['BASIC_AS'] + tt_uppvt_yr['AS_BNR_N'] + tt_uppvt_yr['BDENIT'] + tt_uppvt_yr['AS_BNR_P'] + tt_uppvt_yr['BIODRY'] + tt_uppvt_yr['BIOGAS_CWNS'] + tt_uppvt_yr['BNIT'] + tt_uppvt_yr['BNR'] + tt_uppvt_yr['BS_LAGOON'] + tt_uppvt_yr['CHEM-P'] + tt_uppvt_yr['DISINF-O3'] + tt_uppvt_yr['FBI'] + tt_uppvt_yr['LAGOON'] + tt_uppvt_yr['LAGOON_AER'] + tt_uppvt_yr['LAGOON_ANAER'] + tt_uppvt_yr['LAGOON_FAC'] + tt_uppvt_yr['LAND_TRT'] + tt_uppvt_yr['LIME'] + tt_uppvt_yr['MBR-BNR'] + tt_uppvt_yr['MHI'] + tt_uppvt_yr['NIT'] + tt_uppvt_yr['STBL_POND'] + tt_uppvt_yr['TF_ALL']

  return tt_uppvt_yr

In [ ]:
def treatment_train_werf(tt_upadd_yr, bgyr):
  '''
  Function that assigns wastewater treatment facilities with enough unit process information one or more treatment trains
    Parameters:
      tt_upadd_yr = reported unit processes for each facility in pivot table form
      bgyr = year which biogas systems for electricity generation are confirmed to be active
    Returns:
      tt_werf_yr = dataframe with treatment train assignments for each facility that has sufficient unit process information
  '''

  tt_werf_yr = tt_upadd_yr

  def assign(name, check, exceptions=[]):
    '''
    Core function which assigns a treatment train if a subset of key unit processes has been reported in CWNS
      Parameters:
        name: treatment train to be assigned, written in Tarallo et al., 2015 naming convention
        check: unit processes that have to be present in order for treatment train to be assigned
        exceptions: unit processes and/or treatment trains that must be absent/not yet assigned in order for treatment train to be assigned
      Returns:
        tt_werf_yr = dataframe with treatment train assignments for each facility that has sufficient unit process information
    '''
    #sum the number of key unit processes present within check for a given treatment train
    #for given treatment train, set value equal to the sum above
    tt_werf_yr[name] = sum(tt_werf_yr[check[i]] for i in range(len(check)))

    #if sum of relevant unit processes is not equal to the number of key unit processes in check, turn off treatment train
    tt_werf_yr.loc[tt_werf_yr[name] != len(check), name] = 0

    #if sum of relevant unit processes is equal to the number of key unit processes in check, turn on treatment train
    tt_werf_yr.loc[tt_werf_yr[name] == len(check), name] = 1

    #if exceptions exist, iterate through exceptions and turn treatment train off if one or more exceptions is met
    if len(exceptions) > 0:
        for exception in exceptions:
            tt_werf_yr.loc[tt_werf_yr[exception] == 1, name] = 0

    return tt_werf_yr

  #check treatment trains sequentially and assign if all key unit processes exist and excepting unit processes/trains do not

  #membrane bioreactor trains
  assign('N1E', ['MBR-BNR','AND',f'BIOGAS_EL_{bgyr}'])
  assign('N1', ['MBR-BNR','AND'],['N1E'])
  assign('N2', ['MBR-BNR','AED'])

  #uncomment code below if ENR is identified in cumulative unit process list
  # assign('M1', ['AS','AND','ENR','CHEM-P']),
  # assign('L1', ['AS','AND','ENR'])

  #biological and chemical phosphorus removal trains- priority 1 within activated sludge assignment
  assign('H1E', ['AS_BNR_P','AND','CHEM-P',f'BIOGAS_EL_{bgyr}'])
  assign('H1', ['AS_BNR_P','AND','CHEM-P'],['H1E'])

  #biological phosphorus removal trains- priority 2 in activated sludge assignment
  #G train not assigned if an H train has already been assigned
  assign('G6', ['AS_BNR_P','FBI'])
  assign('G5', ['AS_BNR_P','MHI'])
  assign('G3', ['AS_BNR_P','LIME'])
  assign('G2', ['AS_BNR_P','AED'])
  assign('G1E', ['AS_BNR_P','AND',f'BIOGAS_EL_{bgyr}'],['H1E','H1'])
  assign('G1', ['AS_BNR_P','AND'], ['G1E','H1E','H1'])

  #biological nitrogen removal trains- priority 3 in assignment
  #I train not assigned if H or G train has already been assigned
  assign('I6', ['AS_BNR_N','FBI'],['G6'])
  assign('I5', ['AS_BNR_N','MHI'],['G5'])
  assign('I3', ['AS_BNR_N','LIME'],['G3'])
  assign('I2', ['AS_BNR_N','AED'],['G2'])
  assign('I1E', ['AS_BNR_N','AND',f'BIOGAS_EL_{bgyr}'],['H1','H1E','G1','G1E'])
  assign('I1', ['AS_BNR_N','AND'], ['I1E','H1','H1E','G1','G1E'])

  #nitrification trains- priority 3 in assignment
  #F/E trains not assigned if H, G, or I train has already been assigned
  assign('F1E', ['BASIC_AS','AND','NIT',f'BIOGAS_EL_{bgyr}'], ['AS_BNR_N','G1','G1E','H1','H1E','I1','I1E'])
  assign('F1', ['BASIC_AS','AND','NIT'], ['AS_BNR_N','F1E','G1','G1E','H1','H1E','I1','I1E'])
  assign('E2P', ['BASIC_AS','AED','NIT','PRIMARY'], ['AS_BNR_N','G2','I2'])
  assign('E2', ['BASIC_AS','AED','NIT'], ['AS_BNR_N','G2','I2','E2P'])

  #pure oxygen activated sludge trains- priority 4 in activated sludge assignment
  #O train not assigned if E, F, H, G, or I train has already been assigned
  assign('O5', ['AS-PUREO','MHI'],['G5','I5'])
  assign('O6', ['AS-PUREO','FBI'],['G6','I6'])
  assign('O3', ['AS-PUREO','LIME'],['G3','I3'])
  assign('O2', ['AS-PUREO','AED'],['G2','I2','E2','E2P'])
  assign('O1E', ['AS-PUREO','AND',f'BIOGAS_EL_{bgyr}'],['F1','F1E','G1E','G1','I1','I1E','H1','H1E'])
  assign('O1', ['AS-PUREO','AND'], ['F1','F1E','O1E','G1E','G1','I1','I1E','H1','H1E'])

  #trickling filter trains
  #D train assignment can exist in multiple treatment trains alongside activated sludge systems
  assign('D5', ['TF_ALL','MHI'])
  assign('D6', ['TF_ALL','FBI'])
  assign('D3', ['TF_ALL','LIME'])
  assign('D2', ['TF_ALL','AED'])
  assign('D1E', ['TF_ALL','AND',f'BIOGAS_EL_{bgyr}'])
  assign('D1', ['TF_ALL','AND'],['D1E'])

  #basic activated sludge, primary trains- priority 5 in activated sludge assignment
  #B train not assigned if O, E, F, H, G, or I train has already been assigned
  assign('B6', ['BASIC_AS','FBI','PRIMARY'], ['G6','I6','O6'])
  assign('B5', ['BASIC_AS','MHI','PRIMARY'], ['AS-PUREO','G5','I5','O5'])
  assign('B4', ['BASIC_AS','AND','BIODRY','PRIMARY'])
  assign('B3', ['BASIC_AS','LIME','PRIMARY'], ['G3','I3','O3'])
  assign('B2', ['BASIC_AS','AED','PRIMARY'], ['E2','E2P','G2','I2','N2','O2'])
  assign('B1E', ['BASIC_AS','AND','PRIMARY',f'BIOGAS_EL_{bgyr}'],['AS_BNR_N','AS-PUREO','B4','F1','F1E','G1','G1E','H1','H1E','I1','I1E','N1','N1E','O1','O1E'])
  assign('B1', ['BASIC_AS','AND','PRIMARY'], ['AS_BNR_N','AS-PUREO','B1E','B4','F1','F1E','G1','G1E','H1','H1E','I1','I1E','N1','N1E','O1','O1E'])

  #basic activated sludge trains- priority 6 in activated sludge assignment
  #B train not assigned if B, O, E, F, H, G, or I train has already been assigned
  assign('C5', ['BASIC_AS','MHI'], ['B5','G5','I5','O5'])
  assign('C6', ['BASIC_AS','FBI'], ['B6','G6','I6','O6'])
  assign('C3', ['BASIC_AS','LIME'], ['B3','G3','I3','O3'])
  assign('C2', ['BASIC_AS','AED'], ['B2','E2','E2P','G2','I2','N2','O2'])
  assign('C1E', ['BASIC_AS','AND',f'BIOGAS_EL_{bgyr}'], ['B1','B1E','B4','F1','F1E','G1','G1E','H1','H1E','I1E','I1','N1E','N1','O1','O1E'])
  assign('C1', ['BASIC_AS','AND'], ['B1','B1E','B4','C1E','F1','F1E','G1','G1E','H1','H1E','I1','I1E','N1','N1E','O1','O1E'])

  #identify the number of treatment trains assigned for each facility in the first round of assignment
  tt_werf_yr['TT_IDENTIFIED'] = sum(tt_werf_yr[i] for i in ('LAGOON','LAGOON_AER','LAGOON_ANAER','LAGOON_FAC',
                                                            'STBL_POND','I1E','G6','I6',
                                                            'O5','O6','O3','O1E','G5',
                                                            'I5','C5','C6','O2','O1',
                                                            'N1','N1E','N2','I3','I2','I1',
                                                            'H1','H1E','G3','G2','G1','G1E',
                                                            'F1','F1E','E2','E2P','D5','D6',
                                                            'D1','D1E','D3','D2','C3','C2',
                                                            'C1','C1E','B6','B5','B4','B3',
                                                            'B1E','B1','B2'))

  #for treatment trains that were identified using reported unit processes, add a note
  tt_werf_yr.loc[tt_werf_yr['TT_IDENTIFIED'] > 0, 'TT_ASSIGN_NOTE'] = 'Assigned based on reported unit processes'

  return tt_werf_yr

# Assign treatment trains

In [ ]:
#filter unit process list to unit processes that were present in 2022- includes plants with reported processes from 2004, 2008, 2012, and 2022 CWNS
uplist_werf_2022 = uplist_eicodes[['CWNS_NUM','REPORT_YEAR','2022_MIN_IND','WERF_CODE']]
uplist_werf_2022 = uplist_werf_2022.loc[(uplist_werf_2022['2022_MIN_IND'] == 1)]
uplist_werf_2022 = uplist_werf_2022.dropna(subset = ['WERF_CODE'])
uplist_werf_2022.sort_values(by = ['CWNS_NUM','REPORT_YEAR'], ascending = True, inplace = True, ignore_index = True)

#retain only most recently reported secondary/solids processes
uplist_werf_2022_final = clear_old_treatment(uplist_werf_2022, '2022_MIN')

#create a pivot table describing all the unit processes at a certain wwtp in 2022
tt_uppvt_2022 = unit_process_pivot(uplist_werf_2022_final,'2022_MIN')

<ipython-input-110-a80931cb9198>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uplist_yr_table_dig['DUP'] = uplist_yr_table_dig.duplicated(subset = 'CWNS_NUM', keep = False)
<ipython-input-111-1898439fa410>:12: FutureWarning: The provided callable <function sum at 0x7f129510cf40> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  tt_uppvt_yr = pd.pivot_table(uplist_werf_yr_to_pivot, index = 'CWNS_NUM', values = "%s_IND" %scenario, columns = 'WERF_CODE', aggfunc = np.sum, fill_value = 0)
<ipython-input-111-1898439fa410>:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will ch

In [ ]:
#filter main wwtp dataframe to relevant columns
wwtps_trt_2022 = wwtps[['CWNS_NUM','PRES_AMMONIA_REMOVAL','PRES_NITROGEN_REMOVAL','PRES_PHOSPHOROUS_REMOVAL','BIOGAS_EL_2022','FLOW_2022_MGD','2022_FLOW_CAT_MGD','EPA_REGION']]

#merge unit process list and facility info tables; replace nan values with 0
tt_upadd_2022 = pd.merge(left = tt_uppvt_2022, right = wwtps_trt_2022, how = 'right', on = 'CWNS_NUM')
tt_upadd_2022 = tt_upadd_2022.fillna(0)

#use the ammonia/nitrogen/phosphorous removal flags provided in the 2004, 2008, and 2012 CWNS to add nutrient removal processes to unit process list
#NIT_FLAG column is just used for tracking which nitrification unit processes were added via the PRES_AMMONIA_REMOVAL column
tt_upadd_2022.loc[tt_upadd_2022['PRES_AMMONIA_REMOVAL'] == 'Y', "NIT"] = 1
tt_upadd_2022.loc[tt_upadd_2022['PRES_AMMONIA_REMOVAL'] == 'Y', "NIT_FLAG"] = 1
tt_upadd_2022.loc[tt_upadd_2022['PRES_NITROGEN_REMOVAL'] == 'Y', 'BNR'] = 1
tt_upadd_2022.loc[tt_upadd_2022['PRES_PHOSPHOROUS_REMOVAL'] == 'Y', 'P_REMOVAL'] = 1

#assume all phosphorus removal not specified as biological is done using chemical processes
tt_upadd_2022.loc[(tt_upadd_2022['AS_BNR_P'] == 0) & (tt_upadd_2022['BIO-P'] == 0) & (tt_upadd_2022['P_REMOVAL'] == 1), 'CHEM-P'] = 1

#filter to relevant columns
tt_upadd_2022 = tt_upadd_2022[['CWNS_NUM','AED','AND','AS','AS-A2O','AS-BDENIT','AS-EA','AS-OD','AS-P','AS-PUREO','AS-SA','AS-SBR','BDENIT','BIO-P','BIODRY','BIOGAS_CWNS','BNIT','BNR','BS_LAGOON','CHEM-P','DISINF-O3','FBI','LAGOON','LAGOON_AER','LAGOON_ANAER','LAGOON_FAC','LAND_TRT','LIME','MBR-BNR','MHI','NIT','NIT_FLAG','PRIMARY','STBL_POND','TF','TF-BF','TF-RBC','SUM_AS','TF_ALL','BASIC_AS','AS_BNR_N','AS_BNR_P','COUNT_UP','BIOGAS_EL_2022','FLOW_2022_MGD','2022_FLOW_CAT_MGD','EPA_REGION']]

In [ ]:
#for wwtps with sufficient unit process data, assign treatment trains and filter to relevant columns
ttwerf_2022 = treatment_train_werf(tt_upadd_2022, 2022)
ttwerf_2022 = ttwerf_2022.loc[:,['CWNS_NUM','AED','AND','AS','AS-A2O','AS-BDENIT','AS-EA','AS-OD','AS-P','AS-PUREO','AS-SA','AS-SBR','BDENIT','BIO-P','BIODRY','BIOGAS_CWNS','BNIT','BNR','BS_LAGOON','CHEM-P','DISINF-O3','FBI','LAGOON','LAGOON_AER','LAGOON_ANAER','LAGOON_FAC','LAND_TRT','LIME','MBR-BNR','MHI','NIT','NIT_FLAG','PRIMARY','STBL_POND','TF','TF-BF','TF-RBC','SUM_AS','TF_ALL','BASIC_AS','AS_BNR_N','AS_BNR_P','COUNT_UP','BIOGAS_EL_2022','FLOW_2022_MGD','2022_FLOW_CAT_MGD','EPA_REGION','C1','C1E','C2','C3','C5','C6','B1','B1E','B2','B3','B4','B5','B6','D1','D1E','D2','D3','D5','D6','E2','E2P','F1','F1E','G1','G1E','G2','G3','G5','G6','H1','H1E','I1','I1E','I2','I3','I5','I6','N1','N1E','N2','O1','O1E','O2','O3','O5','O6','TT_IDENTIFIED','TT_ASSIGN_NOTE']]

#for treatment O1 trains that have nitrification, switch to F1
index = ttwerf_2022.loc[(ttwerf_2022['O1'] == 1) & (ttwerf_2022['NIT'] == 1)].index
for i in index:
  ttwerf_2022.at[i,'F1'] = 1
  ttwerf_2022.at[i,'O1'] = 0

#for treatment O1E trains that have nitrification, switch to F1E
index2 = ttwerf_2022.loc[(ttwerf_2022['O1E'] == 1) & (ttwerf_2022['NIT'] == 1)].index
for i in index2:
  ttwerf_2022.at[i,'F1E'] = 1
  ttwerf_2022.at[i,'O1E'] = 0

In [ ]:
#check for facilities that report both a secondary and solids treatment process that have not yet been assigned a treatment train
tt_werf_2022_noid = ttwerf_2022.loc[(ttwerf_2022['TT_IDENTIFIED'] == 0) & (ttwerf_2022['COUNT_UP'] > 0)]
tt_werf_2022_noid['ID_SECTRT'] = tt_werf_2022_noid['SUM_AS'] + tt_werf_2022_noid['TF_ALL'] + tt_werf_2022_noid['LAGOON']
tt_werf_2022_noid['ID_SOLIDS'] = tt_werf_2022_noid['AED'] + tt_werf_2022_noid['AND'] + tt_werf_2022_noid['LIME'] + tt_werf_2022_noid['MHI'] + tt_werf_2022_noid['FBI']
tt_werf_2022_noid2 = tt_werf_2022_noid.loc[(tt_werf_2022_noid['ID_SECTRT'] > 0) | (tt_werf_2022_noid['ID_SOLIDS'] > 0)]

#assert statement failure indicates treatment_train_werf() function is not working correctly
assert tt_werf_2022_noid2.loc[(tt_werf_2022_noid2['ID_SECTRT'] > 0) & (tt_werf_2022_noid2['ID_SOLIDS'] > 0)].shape[0] == 0

<ipython-input-159-56f5a5ce0012>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tt_werf_2022_noid['ID_SECTRT'] = tt_werf_2022_noid['SUM_AS'] + tt_werf_2022_noid['TF_ALL'] + tt_werf_2022_noid['LAGOON']
<ipython-input-159-56f5a5ce0012>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tt_werf_2022_noid['ID_SOLIDS'] = tt_werf_2022_noid['AED'] + tt_werf_2022_noid['AND'] + tt_werf_2022_noid['LIME'] + tt_werf_2022_noid['MHI'] + tt_werf_2022_noid['FBI']


In [ ]:
#based on the facilities that were assigned a treatment train based solely on reported unit process information, create a dataframe that contains both the most common treatment train overall and the most common treatment train with a key unit process present for all possible EPA region / plant size combinations. This dataframe is later used to make treatment train assignments for facilities with limited unit process information.

#create a dataframe of all the wwtps that were assigned a treatment train based purely on reported unit processes
assigned = ttwerf_2022.loc[ttwerf_2022['TT_IDENTIFIED'] != 0]
assigned.reset_index(inplace = True, drop = True)

#create combined columns for 1 and 1E trains
assigned['B1B1E'] = assigned['B1'] + assigned['B1E']
assigned['C1C1E'] = assigned['C1'] + assigned['C1E']
assigned['D1D1E'] = assigned['D1'] + assigned['D1E']
assigned['F1F1E'] = assigned['F1'] + assigned['F1E']
assigned['G1G1E'] = assigned['G1'] + assigned['G1E']
assigned['H1H1E'] = assigned['H1'] + assigned['H1E']
assigned['I1I1E'] = assigned['I1'] + assigned['I1E']
assigned['N1N1E'] = assigned['N1'] + assigned['N1E']
assigned['O1O1E'] = assigned['O1'] + assigned['O1E']

#filter to relevant columns
assigned = assigned[['2022_FLOW_CAT_MGD', 'EPA_REGION', 'LAGOON','LAGOON_AER','LAGOON_ANAER','LAGOON_FAC','STBL_POND','C1C1E', 'C2', 'C3', 'C5',
       'C6', 'B1B1E', 'B2', 'B3', 'B4', 'B5', 'B6', 'D1D1E', 'D2', 'D3', 'D5',
       'D6', 'E2', 'E2P', 'F1F1E','G1G1E', 'G2', 'G3', 'G5', 'G6', 'H1H1E',
       'I1I1E', 'I2', 'I3', 'I5', 'I6', 'N1N1E','N2', 'O1O1E', 'O2',
       'O3', 'O5', 'O6']]

#group assigned trains by EPA region and plant size
most_common = assigned.groupby(['2022_FLOW_CAT_MGD', 'EPA_REGION']).sum()

#create fields for the most common treatment train per EPA region / plant size overall and for just trains with key unit processes present
most_common['Most Common TT (OVERALL)'] = np.nan
most_common['Most Common TT (BASIC_AS)'] = np.nan
most_common['Most Common TT (AS_BNR_N)'] = np.nan
most_common['Most Common TT (AS-PUREO)'] = np.nan
most_common['Most Common TT (AND)'] = np.nan
most_common['Most Common TT (AED)'] = np.nan
most_common['Most Common TT (LIME)'] = np.nan
most_common['Most Common TT (FBI)'] = np.nan
most_common['Most Common TT (MHI)'] = np.nan
most_common['Most Common TT (TF_ALL)'] = np.nan
most_common['Most Common TT (NIT)'] = np.nan
most_common['Most Common TT (AS_BNR_P)'] = np.nan

#define list of key unit processes and the treatment trains they are present within
key_ups = {'BASIC_AS':['F1F1E','E2P','E2','B6','B5','B4','B3','B2','B1B1E','C5','C6','C3','C2','C1C1E'],'AS_BNR_N':['H1H1E','G6','G5','G3','G2','G1G1E'],'AS-PUREO':['O5','O6','O3','O2','O1O1E'],'AND':['O1O1E','N1N1E','H1H1E','G1G1E','I1I1E','F1F1E','D1D1E','B1B1E','C1C1E'],'AED':['O2','N2','G2','I2','E2P','E2','D2','B2','C2'],'LIME':['O3','G3','I3','D3','B3','C3'],'FBI':['O6','G6','I6','D6','B6','C6'],'MHI':['O5','G5','I5','D5','B5','C5'],'TF_ALL':['D5','D6','D3','D1D1E'],'NIT':['E2','E2P','F1F1E'], 'AS_BNR_P':['I6','I5','I3','I2','I1I1E']}

#iterate through each EPA region / plant size combination
for index, row in most_common.iterrows():
  #identify most common treatment train for region / plant size
  if row.max() != 0:
    #if there are no ties for most common treatment train
    if (row == row.max()).sum() == 1:
      most_common.at[index, 'Most Common TT (OVERALL)'] = row[row == row.max()].index.values[0]
    #if there are ties for the most common treatment train, separate with a slash
    else:
      most_common.at[index, 'Most Common TT (OVERALL)'] = '/'.join(row[row == row.max()].index.values.tolist())
  #iterate through key unit processes
  for key_up in key_ups:
    #identify treatment trains that contain that key unit process
    relevant_tts = key_ups[key_up]
    #identify the most common treatment train within relevant subset
    if row[relevant_tts].max() != 0:
      #if there are no ties for most common treatment train
      if (row[relevant_tts] == row[relevant_tts].max()).sum() == 1:
        most_common.at[index, ('Most Common TT (' + key_up +')')] = row[relevant_tts][row[relevant_tts] == row[relevant_tts].max()].index.values[0]
      #if there are ties for the most common treatment train, separate with a slash
      else:
        most_common.at[index, ('Most Common TT (' + key_up +')')] = '/'.join(row[relevant_tts][row[relevant_tts] == row[relevant_tts].max()].index.values.tolist())

#if there are no facilities w/ key nutrient removal process in the size/region combination, override with most common treatment train containing key unit process across all EPA regions and size categories
for index, row in most_common.iterrows():
  for key_up in ['AS_BNR_P','NIT','AS_BNR_N']:
    if pd.isna(row['Most Common TT (' + key_up +')']):
      lst = list(most_common['Most Common TT (' + key_up +')'].dropna().values)
      most_common.at[index, 'Most Common TT (' + key_up +')'] = max(set(lst), key=lst.count)

#in the "Most Common TT" columns, replace B1B1E with B1, C1C1E with C1, etc. to ensure that electricity-producing trains are not assigned as most common trains
most_common = most_common.replace({'B1B1E':'B1','C1C1E':'C1','D1D1E': 'D1','F1F1E': 'F1','G1G1E': 'G1','H1H1E': 'H1','I1I1E': 'I1','N1N1E':'N1','O1O1E':'O1'}, regex=True)

<ipython-input-160-a4f9f878e62e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assigned['B1B1E'] = assigned['B1'] + assigned['B1E']
<ipython-input-160-a4f9f878e62e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assigned['C1C1E'] = assigned['C1'] + assigned['C1E']
<ipython-input-160-a4f9f878e62e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [ ]:
#for unassigned treatment trains that contain key unit processes ('BASIC_AS, 'AS_BNR_N','AS_BNR_P','AS_PUREO','AND','AED','LIME','FBI','MHI','TF_ALL',and 'NIT'), assign a treatment train based on the most common train for that size/region that includes that key unit process

#identify wwtps that do not yet have an assigned treatment train
no_tt = ttwerf_2022.loc[ttwerf_2022['TT_IDENTIFIED'] == 0].reset_index(drop = True)

#define list of key unit processes
key_ups = ['BASIC_AS','AS_BNR_N','AS_BNR_P','AS-PUREO','AND','AED','LIME','FBI','MHI','TF_ALL','NIT']

#for specific unit processes (AS_BNR_P, AS_BNR_N, NIT, AS_PUREO, BASIC_AS), if one is present, turn off less important unit processes
#e.g., if AS_BNR_P is present, turn off AS_BNR_N, NIT, AS_PUREO, BASIC_AS; if AS_BNR_N is present, turn off NIT, AS_PUREO, BASIC_AS, etc.
for index, row in no_tt.iterrows():
  if row['AS_BNR_P'] == 1:
    no_tt.at[index, 'AS_BNR_N'] = 0
    no_tt.at[index, 'NIT'] = 0
    no_tt.at[index, 'AS-PUREO'] = 0
    no_tt.at[index, 'BASIC_AS'] = 0
  elif row['AS_BNR_N'] == 1:
    no_tt.at[index, 'NIT'] = 0
    no_tt.at[index, 'AS-PUREO'] = 0
    no_tt.at[index, 'BASIC_AS'] = 0
  elif row['NIT'] == 1:
    no_tt.at[index, 'AS-PUREO'] = 0
    no_tt.at[index, 'BASIC_AS'] = 0
  elif row['AS-PUREO'] == 1:
    no_tt.at[index, 'BASIC_AS'] = 0

#iterate through wwtps without an assigned treatment train
for index, row in no_tt.iterrows():
  #identify size and EPA region of current wwtp
  size = row['2022_FLOW_CAT_MGD']
  region = row['EPA_REGION']
  #check for key unit processes in current wwtp
  for up in key_ups:
    if row[up] > 0:
      #identify most common treatment train for plants of a similar size, region, and key unit process
      tt_common = most_common.at[(size,region),('Most Common TT (' + up + ')')]
      #turn on treatment train in no_tt dataframe
      if pd.isna(tt_common) == False:
        #if only one treatment train was identified
        if '/' not in tt_common:
          #if most common treatment train is a conventional activated sludge train, but the nitrification flag is on, override tt_common to nan
          if (row['NIT'] == 1) & (tt_common[0] == 'B' or tt_common[0] == 'C'):
            tt_common = np.nan
          else:
            no_tt.at[index, tt_common] = 1
            no_tt.at[index, 'TT_IDENTIFIED'] = no_tt.at[index,'TT_IDENTIFIED'] + 1
            no_tt.at[index, 'TT_ASSIGN_NOTE'] = 'Assigned based on partial unit process information based on common treatment trains of similar size in EPA region'
        #if multiple treatment trains were identified, split into multiple strings before turning on treatment trains in no_tt dataframe
        else:
          tt_common_multiple = tt_common.split('/')
          for tt in tt_common_multiple:
            #if most common treatment train is a conventional activated sludge train, but the nitrification flag is on, override tt_common to nan
            if (row['NIT'] == 1) & (tt[0] == 'B' or tt[0] == 'C'):
              tt_common = np.nan
            else:
              no_tt.at[index, tt] = 1
              no_tt.at[index, 'TT_IDENTIFIED'] = no_tt.at[index,'TT_IDENTIFIED'] + 1
              no_tt.at[index, 'TT_ASSIGN_NOTE'] = 'Assigned based on partial unit process information based on common treatment trains of similar size in EPA region'

#add original unit processes back into the no_tt dataframe
original_ups = ttwerf_2022.loc[ttwerf_2022['TT_IDENTIFIED'] == 0].reset_index(drop = True)[['CWNS_NUM', 'AED', 'AND', 'AS', 'AS-A2O', 'AS-BDENIT',
       'AS-EA', 'AS-OD', 'AS-P', 'AS-PUREO', 'AS-SA', 'AS-SBR', 'BDENIT',
       'BIO-P', 'BIODRY', 'BIOGAS_CWNS', 'BNIT', 'BNR', 'BS_LAGOON', 'CHEM-P',
       'DISINF-O3', 'FBI', 'LAGOON', 'LAGOON_AER', 'LAGOON_ANAER',
       'LAGOON_FAC', 'LAND_TRT', 'LIME', 'MBR-BNR', 'MHI', 'NIT', 'NIT_FLAG',
       'PRIMARY', 'STBL_POND', 'TF', 'TF-BF', 'TF-RBC', 'SUM_AS', 'TF_ALL',
       'BASIC_AS', 'AS_BNR_N', 'AS_BNR_P']]

new_tts = no_tt[['COUNT_UP', 'BIOGAS_EL_2022',
       'FLOW_2022_MGD', '2022_FLOW_CAT_MGD', 'EPA_REGION', 'C1', 'C1E', 'C2',
       'C3', 'C5', 'C6', 'B1', 'B1E', 'B2', 'B3', 'B4', 'B5', 'B6', 'D1',
       'D1E', 'D2', 'D3', 'D5', 'D6', 'E2', 'E2P', 'F1', 'F1E', 'G1', 'G1E',
       'G2', 'G3', 'G5', 'G6', 'H1', 'H1E', 'I1', 'I1E', 'I2', 'I3', 'I5',
       'I6', 'N1', 'N1E', 'N2', 'O1', 'O1E', 'O2', 'O3', 'O5', 'O6',
       'TT_IDENTIFIED', 'TT_ASSIGN_NOTE']]

no_tt = pd.concat([original_ups, new_tts], axis = 1)

#merge new assignments based on partial unit process information back into main dataframe
ttwerf_2022 = ttwerf_2022.loc[ttwerf_2022['TT_IDENTIFIED'] != 0]
ttwerf_2022 = pd.concat([ttwerf_2022, no_tt], axis = 0)

In [ ]:
#for remaining wwtps without a treatment train assignment, assign a treatment train using the most common treatment train of that size/region (ignoring production of energy from biogas)
no_tt = ttwerf_2022.loc[ttwerf_2022['TT_IDENTIFIED'] == 0].reset_index(drop = True)

#iterate through wwtps without an assigned treatment train
for index, row in no_tt.iterrows():
  #identify size and EPA region of current wwtp
  size = row['2022_FLOW_CAT_MGD']
  region = row['EPA_REGION']
  #identify most common treatment train for plants of a similar size and region
  tt_common = most_common.at[(size,region),('Most Common TT (OVERALL)')]
  #turn on treatment train in no_tt dataframe
  if pd.isna(tt_common) == False:
    #if only one treatment train was identified
    if '/' not in tt_common:
      #if most common treatment train is a conventional activated sludge train, but the nitrification flag is on, override tt_common to nan
      if (row['NIT'] == 1) & (tt_common[0] == 'B' or tt_common[0] == 'C'):
        tt_common = np.nan
      else:
        no_tt.at[index, tt_common] = 1
        no_tt.at[index, 'TT_IDENTIFIED'] = no_tt.at[index,'TT_IDENTIFIED'] + 1
        no_tt.at[index, 'TT_ASSIGN_NOTE'] = 'Assigned based on common treatment trains of similar size in EPA region'
    #if multiple treatment trains were identified, split into multiple strings before turning on treatment trains in no_tt dataframe
    else:
      tt_common_multiple = tt_common.split('/')
      for tt in tt_common_multiple:
        #if most common treatment train is a conventional activated sludge train, but the nitrification flag is on, override tt_common to nan
        if (row['NIT'] == 1) & (tt[0] == 'B' or tt[0] == 'C'):
          tt = np.nan
        else:
          no_tt.at[index, tt] = 1
          no_tt.at[index, 'TT_IDENTIFIED'] = no_tt.at[index,'TT_IDENTIFIED'] + 1
          no_tt.at[index, 'TT_ASSIGN_NOTE'] = 'Assigned based on common treatment trains of similar size in EPA region'

#merge new assignments back into main dataframe
ttwerf_2022 = ttwerf_2022.loc[ttwerf_2022['TT_IDENTIFIED'] != 0]
ttwerf_2022 = pd.concat([ttwerf_2022, no_tt], axis = 0)

In [ ]:
#update the UP_ID_NOTE column to reflect manual unit process checks
tt_werf_2022_final = pd.merge(left = ttwerf_2022, right = manual_check_ups, how = 'left', on = 'CWNS_NUM')

In [ ]:
#identify facilities that reported using biogas to generate electricity in the DOE or WEF databases that were not assigned an electricity-generating treatment train
biogas_tts = tt_werf_2022_final.loc[tt_werf_2022_final['BIOGAS_EL_2022'] == 1]
biogas_tts.reset_index(inplace = True, drop = True)
for wwtp in biogas_tts['CWNS_NUM']:
   if biogas_tts.loc[biogas_tts['CWNS_NUM'] == wwtp][['C1E','B1E','D1E','F1E','H1E','N1E','I1E','G1E','O1E']].values.sum() > 0:
      biogas_tts.loc[biogas_tts['CWNS_NUM'] == wwtp, 'Assigned an E train?'] = 'Yes'
   else:
      biogas_tts.loc[biogas_tts['CWNS_NUM'] == wwtp, 'Assigned an E train?'] = 'No'

#create a column that contains which treatment train(s) a facility was assigned
tts = ['LAGOON', 'LAGOON_AER','LAGOON_ANAER', 'LAGOON_FAC', 'STBL_POND', 'C1', 'C1E','C2', 'C3', 'C5', 'C6','B1', 'B1E', 'B2', 'B3', 'B4', 'B5', 'B6', 'D1', 'D1E','D2', 'D3', 'D5', 'D6','E2', 'E2P', 'F1', 'F1E','I1', 'I1E', 'I2', 'I3', 'I5', 'I6', 'G1', 'G1E','G2', 'G3', 'G5', 'G6', 'H1', 'H1E','N1', 'N1E','N2', 'O1', 'O1E', 'O2', 'O3', 'O5','O6']
biogas_tts['TT_ASSIGNED'] = np.nan
biogas_tts['TT_ASSIGNED'] = biogas_tts['TT_ASSIGNED'].astype('object')
for row in range(0,biogas_tts.shape[0]):
  tt_curr = []
  for tt in tts:
    if biogas_tts.iloc[row][tt] == 1:
      tt_curr = tt_curr + [tt]
  biogas_tts.at[row,'TT_ASSIGNED'] = tt_curr

#for facilities that reported electricity generation but were not assigned an "E" train, override to the electricity-producing version of the treatment train
override = biogas_tts.loc[((biogas_tts['TT_ASSIGN_NOTE'] == 'Assigned based on partial unit process information based on common treatment trains of similar size in EPA region') | (biogas_tts['TT_ASSIGN_NOTE'] == 'Assigned based on common treatment trains of similar size in EPA region')) & (biogas_tts['Assigned an E train?'] == 'No')].reset_index(drop = True)
for index, row in override.iterrows():
  tts = row['TT_ASSIGNED']
  #if just one treatment train identified, switch to electricity-producing version of that train
  if len(tts) == 1:
    if tts[0] != 'E2P':
      override.at[index, tts[0]] = 0
      override.at[index, tts[0] + 'E'] = 1
  #if multiple treatment trains were identified, switch to electricity-producing version of those trains
  else:
    for tt in tts:
      if tts[0] != 'E2P':
        override.at[index, tt] = 0
        override.at[index, tt + 'E'] = 1

#add new electricity-producing trains into main dataframe
override.drop(columns = ['Assigned an E train?','TT_ASSIGNED'], inplace = True)
tt_werf_2022_final = tt_werf_2022_final[~tt_werf_2022_final['CWNS_NUM'].isin(override['CWNS_NUM'])]
tt_werf_2022_final = pd.concat([tt_werf_2022_final, override], axis = 0)

<ipython-input-164-6d9613882418>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biogas_tts.loc[biogas_tts['CWNS_NUM'] == wwtp, 'Assigned an E train?'] = 'Yes'
<ipython-input-164-6d9613882418>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biogas_tts['TT_ASSIGNED'] = np.nan
<ipython-input-164-6d9613882418>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [ ]:
#for facilities that reported using biogas to generate electricity, but were not assigned an electricity-producing train due to a lack of unit process information, manually correct treatment train assignment based on publicly available information
#treatment train dataframe is corrected rather than the cumulative unit process list because it is simpler to identify facilities that need biogas corrections after treatment train assignment rather than before

#Kitsap Co SD #7 WWTP; online checks show that wwtp has primary, activated sludge, anaerobic digestion, and nitrification
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '53002625501', 'STBL_POND'] = 0
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '53002625501', 'F1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '53002625501', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#NEW HAVEN EAST SHORE WPCF; online checks show that wwtp generates energy using incinerator
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '09000930003', 'B1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '09000930003', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#STEPHENSON WTP; online checks show that wwtp uses a trickling filter
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '26002047001', 'STBL_POND'] = 0
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '26002047001', 'D1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '26002047001', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#BUCKLIN PT STP; online checks show that wwtp has nitrogen removal and activated sludge
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '44000031001', 'LAGOON'] = 0
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '44000031001', 'I1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '44000031001', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#POCATELLO STP; online checks show that wwtp has ammonia and phosphorous removal
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '16000001001', 'LAGOON_AER'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '16000001001', 'G1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '16000001001', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#Riverside WPCF; online checks show that wwtp uses membrane bioreactors
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '06008001001', 'STBL_POND'] = 0
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '06008001001', 'N1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '06008001001', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#Springfield SW WWTP; online checks show that wwtp uses chemical and biological phosphorus removal
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '29001026001', 'E2P'] = 0
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '29001026001', 'H1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '29001026001', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024))'

#RM Clayton WRP; online checks show that  wwtp uses membrane bioreactors
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '13000012004', 'LAGOON_ANAER'] = 0
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '13000012004', 'N1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '13000012004', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#Landis Sewerage Authority - CS/STP; online checks show that wwtp has nitrification and anaerobic digestion
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '34005051001', 'STBL_POND'] = 0
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '34005051001', 'F1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '34005051001', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#METROPOLITAN WWTP; online checks show that wwtp generates energy using incinerator
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '27000001001', 'G1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '27000001001', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#Davis, City of WWTP; online checks show that wwtp replaced oxidation pond with activated sludge and digestion
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '06005205001', 'LAGOON_AER'] = 0
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '06005205001', 'B1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '06005205001', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#DELHI TWP WWTP; online checks show that wwtp uses primary treatment, anaerobic digestion, and nitrification
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '26000054001', 'STBL_POND'] = 0
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '26000054001', 'F1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '26000054001', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#South Columbus WRP; online checks show that wwtp uses primary treatment and anaerobic digestion
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '13000051001', 'LAGOON_AER'] = 0
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '13000051001', 'B1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '13000051001', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#Sunnyvale, City of (WPCP); online checks show that wwtp uses ponding process and generates electricity
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '06002008001', 'B1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '06002008001', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#Bakersfield WWTP #2; online checks show that wwtp uses trickling filter
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '06005010001', 'D1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '06005010001', 'LAGOON_AER'] = 0
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '06005010001', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#MILWAUKEE MSD COMBINED - South Shore; online checks show that wwtp uses nitrification
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '55000000052', 'F1'] = 0
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '55000000052', 'F1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '55000000052', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#Lemay WWTP; online checks show that wwtp uses incinerator and anaerobic digestion
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '29001023002', 'B6'] = 0
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '29001023002', 'B1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '29001023002', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

#Bissell Point WWTP; online checks show that wwtp uses incinerator, trickling filter, and digestion
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '29001023001', 'B6'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '29001023001', 'D6'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '29001023001', 'B1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '29001023001', 'D1E'] = 1
tt_werf_2022_final.loc[tt_werf_2022_final['CWNS_NUM'] == '29001023001', 'UP_ID_NOTE'] = 'Corrected based on manual check of facilities that utilize biogas to produce electricity (2024)'

In [ ]:
#rename LAGOON column to indicate that type of lagoon was not specified
tt_werf_2022_final.rename(columns = {'LAGOON':'LAGOON_OTHER'}, inplace = True)

#reset the 'TT_IDENTIFIED' column to account for manual corrections
tt_werf_2022_final['TT_IDENTIFIED'] = tt_werf_2022_final[['LAGOON_OTHER', 'LAGOON_AER', 'LAGOON_ANAER',
       'LAGOON_FAC', 'STBL_POND', 'C1', 'C1E', 'C2', 'C3', 'C5', 'C6', 'B1',
       'B1E', 'B2', 'B3', 'B4', 'B5', 'B6', 'D1', 'D1E', 'D2', 'D3', 'D5', 'D6', 'E2',
       'E2P', 'F1', 'F1E', 'I1', 'I1E', 'I2', 'I3', 'I5', 'I6', 'G1', 'G1E', 'G2',
       'G3', 'G5', 'G6', 'H1', 'H1E', 'N1', 'N1E', 'N2', 'O1', 'O1E', 'O2', 'O3', 'O5', 'O6']].sum(axis = 1)

#for facilities where the unit process nitrification was added via the 'PRES_AMMONIA_REMOVAL' column, add a note to the UP_ID_NOTE column
tt_werf_2022_final.loc[tt_werf_2022_final['NIT_FLAG'] == 1, 'UP_ID_NOTE_2'] = "Nitrification unit process added via 'PRES_AMMONIA_REMOVAL' column (2024)"
tt_werf_2022_final.loc[tt_werf_2022_final['UP_ID_NOTE'] == 'nan', 'UP_ID_NOTE'] = np.nan
tt_werf_2022_final.loc[tt_werf_2022_final['UP_ID_NOTE_2'] == 'nan', 'UP_ID_NOTE_2'] = np.nan

for index, row in tt_werf_2022_final.iterrows():
  if pd.isna(row['UP_ID_NOTE_2']) == False:
    if pd.isna(row['UP_ID_NOTE']) == True:
      tt_werf_2022_final.at[index, 'UP_ID_NOTE'] = row['UP_ID_NOTE_2']
    else:
      tt_werf_2022_final.at[index, 'UP_ID_NOTE'] = row['UP_ID_NOTE'] + '; ' + row['UP_ID_NOTE_2']

#reoder columns
tt_werf_2022_final = tt_werf_2022_final[['CWNS_NUM', 'AED', 'AND', 'AS', 'AS-A2O', 'AS-BDENIT',
       'AS-EA', 'AS-OD', 'AS-P', 'AS-PUREO', 'AS-SA', 'AS-SBR', 'BDENIT',
       'BIO-P', 'BIODRY', 'BIOGAS_CWNS', 'BNIT', 'BNR', 'BS_LAGOON', 'CHEM-P',
       'DISINF-O3', 'FBI', 'LAND_TRT', 'LIME', 'MBR-BNR', 'MHI', 'NIT','PRIMARY',
        'TF', 'TF-BF', 'TF-RBC', 'SUM_AS', 'TF_ALL', 'BASIC_AS',
       'AS_BNR_N', 'AS_BNR_P', 'COUNT_UP', 'BIOGAS_EL_2022', 'FLOW_2022_MGD',
       '2022_FLOW_CAT_MGD', 'EPA_REGION','LAGOON_OTHER', 'LAGOON_AER', 'LAGOON_ANAER',
       'LAGOON_FAC', 'STBL_POND', 'C1', 'C1E', 'C2', 'C3', 'C5', 'C6', 'B1',
       'B1E', 'B2', 'B3', 'B4', 'B5', 'B6', 'D1', 'D1E', 'D2', 'D3', 'D5', 'D6', 'E2',
       'E2P', 'F1', 'F1E', 'I1', 'I1E', 'I2', 'I3', 'I5', 'I6', 'G1', 'G1E', 'G2',
       'G3', 'G5', 'G6', 'H1', 'H1E', 'N1', 'N1E', 'N2', 'O1', 'O1E', 'O2', 'O3', 'O5', 'O6',
       'TT_IDENTIFIED', 'TT_ASSIGN_NOTE', 'UP_ID_NOTE']]

In [ ]:
#read in spreadsheet that contains manually corrected flow rates for top 50 wwtps by flow rate (checks done by Heroda Abera, 2024)
flow_checks = pd.read_excel(/input_data/2022_flow_checks.xlsx')

#add leading zero to CWNS ids with less than 11 digits to ensure correct merge with final dataframe
flow_checks['CWNS_NUM'] = ['0' + str(cwns) if len(str(cwns)) < 11 else str(cwns) for cwns in flow_checks['CWNS_NUM']]

#merge flow checks with final dataframe
tt_werf_2022_final = tt_werf_2022_final.merge(flow_checks, how = 'left', on = ['CWNS_NUM'])

#for wwtps that were not checked for flow rate, assume the flow reported in 2022 is correct
tt_werf_2022_final.loc[pd.isna(tt_werf_2022_final['FLOW_2022_MGD (CHECKED)']),'FLOW_2022_MGD (CHECKED)'] = tt_werf_2022_final.loc[pd.isna(tt_werf_2022_final['FLOW_2022_MGD (CHECKED)']),'FLOW_2022_MGD']
tt_werf_2022_final.rename(columns = {'FLOW_2022_MGD (CHECKED)':'FLOW_2022_MGD_FINAL'}, inplace = True)

#drop duplicates
tt_werf_2022_final = tt_werf_2022_final.drop_duplicates()

In [ ]:
#export to csv to be used for energy and greenhouse gas emissions calculations
tt_werf_2022_final.to_csv(/output_data/tt_assignments_2022.csv', index = False)